# Project Proposal: Predicting Heart Disease

**Group 11**

## Introduction

Heart or cardiovascular disease is known as one of the leading causes of death for people of most racial and ethnic groups in the world<sup>4,11</sup>. This disease refers to any condition that affects the structure or function of the heart. The most common type is coronary heart disease, which occurs when there is narrowing of arteries and leads to heart attacks and chest pains or anginas<sup>5,11</sup>. Thus, how to prevent heart disease and how to accurately predict the probability of heart disease is an urgent and crucial scientific topic. In this project, we will answer the question: **What are the statistical correlations between given covariables, and how do their correlations impact the predictive power of logistic regression, specifically on whether a patient has heart disease or not?**

With the prevalence of machine learning, it is gradually being applied to many industries including disease prediction and diagnosis.<sup>1,7</sup>. Current literature has demonstrated two contradicting results in heart disease prediction models. Some research shows that random forest and K-nearest neighbors produce the greatest accuracy<sup>2,6,7,10</sup> in contrast to the findings that multinomial logistic regression model outperformed other hybrid models<sup>9</sup>. Generally, it seems that logistic regression is not very popular, perhaps for its seemingly lower accuracy levels in comparison to other models, but there is no explanation as to why that is. Despite the lack of support for the strength in accuracy for logistic regression, past research does not necessarily suggest that it is an extremely bad choice. Thus, our main purpose is to seek to improve the accuracy of prediction of heart disease using logistic regression and minimize the amount of diagnostic errors to provide a better quality health case service to patients<sup>1</sup>.

The data set we will be using to answer our question is found in the UCI Machine Learning Repository Heart Disease Data Set<sup>8</sup>. This dataset contains a diagnosis of heart disease for each patient, represented by variable num, some descriptive variables (eg. age), and numerous bodily measurements related to the heart, either categorical (eg. cp) or numerical (eg. chol).  The response variable (i.e. "num") and the 13 covariates are in Appendix I. 

## Exploratory Data Analysis

In [ ]:
#install.packages("ggpubr")
library(tidyverse)
library(GGally)
library(gridExtra)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




Loading in the data (replacing all "?" with NA since the predetermined NA value was "?", and NA is easier to deal with). Also, adding a new column `country` which will help keep track of which hospital the data came from.

In [2]:
# Setting names of columns
names <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num", show_col_types = FALSE)

# Reading all of the data from https://archive.ics.uci.edu/ml/datasets/heart+Disease
cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", col_names = names, na = "?", show_col_types = FALSE) %>% mutate(country = 0)
switzerland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.switzerland.data", col_names = names, na = "?", show_col_types = FALSE) %>% mutate(country = 1)
hungary_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data", col_names = names, na = "?", show_col_types = FALSE) %>% mutate(country = 2)
va_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data", col_names = names, na = "?", show_col_types = FALSE) %>% mutate(country = 0)

ERROR: Error in open.connection(structure(4L, class = c("curl", "connection"), conn_id = <pointer: 0x279>), : Timeout was reached: [archive.ics.uci.edu] Connection timeout after 10001 ms


Counts of NA values in each dataset, and within every column. This allows us to see which columns we should remove early on, and which would likely cause issues in terms of missing data.

In [ ]:
NA_counts <- as.data.frame(cleveland_data[FALSE, ])
x <- substitute(list(cleveland_data, switzerland_data, hungary_data, va_data))

for (i in as.list(x)[-1]) {
    NA_counts[nrow(NA_counts) + 1,] <- map_df(get(i), ~sum(is.na(.x)))
    rownames(NA_counts)[nrow(NA_counts)] <- deparse(i)
}
NA_counts

It looks like the columns `slope`, `ca`, and `thal` all have a large quantity of NA values within three of the four datasets that we are using. For this reason, we chose to drop these three columns because performing imputation with such a small quantity of usable data will lead to inaccurate results, and choosing to omit NA values will delete the rows entirely, making us lose a large number of rows. Additionally, we are choosing to drop `fbs`, `exang`, and `oldpeak` for a similar reason. As such, removing these columns will allow us to keep more observations for our analysis, as well as simplify our model later on by exchanging less features for more data.

In [ ]:
data_tidy_multi <- rbind(cleveland_data, switzerland_data, hungary_data, va_data) %>%
    select(-slope, -ca, -thal, -fbs, -exang, -oldpeak) %>%
    na.omit()

num_counts_multi <- data_tidy_multi %>%
    group_by(num) %>%
    summarize(counts = n(), proportions = counts/nrow(data_tidy_multi))
num_counts_multi

We can see that there is quite a large class imbalance within the dataset, particularly in the cases of more severe heart disease. Since this analysis is mainly focused on finding whether patients have heart disease or not, we are not interested in the severity of their condition, rather whether or not heart disease is present at all. Therefore, we can binarize the output variable by grouping categories 1-4 into a single case (has heart disease).

In [ ]:
data_tidy <- data_tidy_multi %>%
    mutate(num = ifelse(num == 0, 0, 1)) %>%
    mutate(num = as_factor(num), cp = as_factor(cp), sex = as_factor(sex), restecg = as_factor(restecg), country = as_factor(country)) %>%
    select(age, sex, cp, trestbps, chol, restecg, thalach, country, num)

num_counts <- data_tidy %>%
    group_by(num) %>%
    summarize(counts = n(), proportions = counts/nrow(data_tidy))

num_counts
head(data_tidy)

Evidently, the data is now much more balanced. Now that we have fully tidied our data (`data_tidy`), we can begin the variable selection portion of the analysis. First, we can make a covariance matrix between all continuous variables in order to get a preliminary look at whether or not there exist any inter-dependencies between variables (ie. colinearity). 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)

covariance_plot <- data_tidy %>%
    select(age, trestbps, chol, thalach, num) %>%
    ggpairs(aes(colour = num, alpha = 0.5), lower = list(combo = wrap("facethist", binwidth = 2)))

covariance_plot

The covariance plot above shows that the distribution for when `chol` = 1 is skewed. A skewed distribution can bias our predictions and model, thus further investigation into categorial variables, including `chol` has been performed in the form of a boxplot matrix. 

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)

numeric_variables <- colnames(select(data_tidy, where(is.numeric)))
categorical_variables <- colnames(select(data_tidy, where(is.factor)))

index <- 1

for (j in categorical_variables) {
    
    count <- 1
    for (i in numeric_variables) {
        p <- data_tidy %>% ggplot() + 
            geom_boxplot(aes(x = .data[[j]], y = .data[[i]], fill = .data[[j]]))
        
        if (count == 1) {
            p <- p + theme(legend.position = "top")
        } else {
            p <- p + theme(legend.position = "none")
        } 
        if (index != 1) {
            p <- p + 
                theme(axis.title.y=element_blank(),
                axis.text.y=element_blank(),
                axis.ticks.y=element_blank())
        } 
        if (count != 4) {
            p <- p + 
                theme(axis.title.x=element_blank(),
                axis.text.x=element_blank(),
                axis.ticks.x=element_blank())
        } 
        
        assign(paste0("plot", toString(count)), p)
        count <- count + 1
    }
    
    g <- plot_grid(plot1, plot2, plot3, plot4, ncol = 1)
    assign(paste0("g", toString(index)), g)
    index <- index + 1
}

plot_grid(g1, g2, g3, g4, g5, ncol = 5)

Firstly, we can see that in all plots with `chol`, there are many observations with `chol` = 0. Naturally, our tidy data will be prone to outliers as no form of data collection is fully robust, however, upon further inspection of the graphs above, we can see that there are so many values of `chol` equal to 0 that the quartiles of the boxplots get skewed. Cholesterol performs important roles, such as making hormones, thus our bodies require cholesterol<sup>3</sup>. This means that logically, it does not make sense for your body to have a 0 mg/dL of cholesterol. 

Therefore, since this measurement is likely a mistake in the data collection process, and is skewing the distribution (including with the mean and standard deviation) of the variable, we have no choice but to remove all rows with `chol` being 0. Showcasing the new summary table after removing the outliers, we get the following. 

In [ ]:
summary_table <- data_tidy %>%
    mutate(across(age:num, ~as.numeric(as.character(.x)))) %>%
    select(-num) %>%
    pivot_longer(cols = age:country, 
                names_to = "variable", 
                values_to = "value") %>%
    group_by(variable) %>%
    summarize(mean = mean(value), sd = sd(value), max = max(value), min = min(value))

t(summary_table)

## Method

**Reliability of Report**

Our report is trustworthy firstly because it is easily reproducible and the data is open source. Many studies comparing the accuracy of predictive models in diagnosing heart disease have used the UCI Heart Disease data set, in particular the Cleveland one<sup>2,7,9</sup>, which suggests that this data set is quite valid and reliable. Moreover, the data set contains information from 3 countries, perhaps reflecting a better representation of our diverse community, and the data size is large, meaning we are more likely to produce better estimates. Lastly, we use reliable sources from the government and past peer-reviewed literature.

**Plan**

In this project, there are 2 things we expect to achieve: 1. A set of pairs of variables that are highly correlated to each other, and the analysis of their impact to the predictive power of a logistic model, and 2. A model that is able to predict whether a person has heart disease or not. Our results can provide insights into the relationships between different health indicators, and generate a model to make predictions about heart disease. To do so, we will carry out our project in two parts. 

In the first part, we will use a covariance/correlation plot of all explanatory variables to illustrate the relationship and dependencies between each pair of them. This process will end up with a set of pairs of explanatory variables with high correlations (standards will be discussed in detail in the project report). Hence, we can choose a set of explanatory variables (denoted as E)  for further regression and model selection.

In the second part, we will use ridge logistic regression to fit the data, and perform a 10-fold cross-validation to estimate the test error of the models with different tuning parameters lambda, and select the lambda and sets of predictors corresponding to the model with least CV score  (i.e. least estimated test error rate). We will not use Cp, AIC and BIC because we do not have access to the population variance and do not know whether we have an unbiased estimate. By doing so, we are able to find the best (or approximately the best) model that predicts the occurrence of heart disease. 

**Implications**

Being able to produce a model that accurately predicts and diagnoses heart disease can truly benefit patients. It will help identify individuals who have signs of heart disease and prevent further development of the disease, decreasing the number of deaths related to heart disease. Ultimately, an accurate model would help improve the quality of health services for patients. It will have preventative and predictive applications in health care.

# References 

<sup>1</sup> Abdar, M., Książek, W., Acharya, U. R., Tan, R.-S., Makarenkov, V., & Pławiak, P. (2019). A new machine learning technique for an accurate diagnosis of coronary artery disease. Computer Methods and Programs in Biomedicine, 179, 104992. https://doi.org/10.1016/j.cmpb.2019.104992

<sup>2</sup> Akella, A., & Akella, S. (2021). Machine learning algorithms for predicting coronary artery disease: efforts toward an open source solution. Future Science OA, 7(6), FSO698. https://doi.org/10.2144/fsoa-2020-0206

<sup>3</sup> CDC. (2021, January 26). Cholesterol Myths and Facts. Centers for Disease Control and Prevention. https://www.cdc.gov/cholesterol/myths_facts.htm

‌
<sup>4</sup> Centers for Disease Control and Prevention. (2020). Heart disease in the United States
https://www.cdc.gov/heartdisease/facts.htm  

<sup>5</sup> Mayo Clinic. (2021, February 9). Heart disease - Symptoms and causes. Mayo Clinic. https://www.mayoclinic.org/diseases-conditions/heart-disease/symptoms-causes/syc-20353118

<sup>6</sup> Muhammad, L. J., Al-Shourbaji, I., Haruna, A. A., Mohammed, I. A., Ahmad, A., & Jibrin, M. B. (2021). Machine Learning Predictive Models for Coronary Artery Disease. SN Computer Science, 2(5). https://doi.org/10.1007/s42979-021-00731-4

<sup>7</sup> Shah, D., Patel, S., & Bharti, S. K. (2020). Heart Disease Prediction using Machine Learning Techniques. International Journal of Innovative Technology and Exploring Engineering, 9(5), 1456–1460. https://doi.org/10.35940/ijitee.e2862.039520

<sup>8</sup> UCI Machine Learning Repository: Heart Disease Data Set. (2019). Uci.edu. https://archive.ics.uci.edu/ml/datasets/Heart+Disease 

<sup>9</sup> Verma, L., Srivastava, S., & Negi, P. C. (2016). A Hybrid Data Mining Model to Predict Coronary Artery Disease Cases Using Non-Invasive Clinical Data. Journal of Medical Systems, 40(7). https://doi.org/10.1007/s10916-016-0536-z

<sup>10</sup> Wang, C., Zhao, Y., Jin, B., Gan, X., Liang, B., Xiang, Y., Zhang, X., Lu, Z., & Zheng, F. (2021). Development and Validation of a Predictive Model for Coronary Artery Disease Using Machine Learning. Frontiers in Cardiovascular Medicine, 8. https://doi.org/10.3389/fcvm.2021.614204

‌<sup>11</sup> World Health Organization. (2021, June 11). Cardiovascular Diseases (CVDs). Who.int; World Health Organization: WHO. https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds)

# Appendices

## Appendix I


![Table 1: Dataset variable names, types, and descriptions](Project/Variable_table.png)